In [1]:
!/opt/anaconda/envs/env_better/bin/pip3 install folium shapely rasterio bs4 tqdm

In [2]:
import rasterio
import rasterio.mask
import requests
import fiona

In [3]:
from bs4 import BeautifulSoup
import requests
from shapely.geometry import shape
from tqdm import tqdm

url = 'https://store.terradue.com/nextgeoss-change-detection-s1/charpiya/'
ext = 'tif'

def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

vector_fn = 'data/InsideCharpiya.shp'

with fiona.open(vector_fn, 'r') as shapefile:
    shapes = [feature['geometry'] for feature in shapefile]

masked_images = list()
dates = list()

with tqdm(total=19) as pbar:
    for file in listFD(url, ext):
        dates.append(file)
        with rasterio.open(file) as src:
            masked_data, masked_transform = rasterio.mask.mask(src, shapes, crop=True) 
            masked_meta = src.meta
            bbox = shape(shapes[0]).bounds
            img = masked_data[0]
            masked_images.append(img)
            pbar.update(1)
            


100%|██████████| 19/19 [02:30<00:00,  7.92s/it]


In [4]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
from helpers import *

fig, ax = plt.subplots(figsize=(10, 25))

def update(i):
    im_normed = convert2byte(masked_images[i-1],0,1)
    ax.imshow(im_normed, cmap='gray')
    ax.set_title("{}".format(dates[i].rsplit('/',1)[-1]), fontsize=12)
    ax.set_axis_off()


anim = FuncAnimation(fig, update, frames=np.arange(0, 19), interval=500)
anim.save('test.gif', dpi=80, writer='imagemagick')
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.


In [5]:
%%html
<img src="test.gif" />

In [20]:
shape=(392,386)

cummulative_changes = np.zeros(shape).astype(np.uint8)
th=0.2

for i, img in enumerate(masked_images):
    if i == 0:
        continue
    diff = masked_images[i][0:shape[0],0:shape[1]]-masked_images[i-1][0:shape[0],0:shape[1]]
    mask = np.zeros(diff.shape).astype(np.uint8)
    mask_th = np.logical_or(np.absolute(diff)>th,mask)
    cummulative_changes += mask_th.astype(np.uint8)



In [25]:
import folium

m = folium.Map([22.38, 91.4], zoom_start=14)

folium_img = folium.raster_layers.ImageOverlay(
                name='Cummulative changes',
                image=cummulative_changes,
                bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],
                opacity=0.7)
folium_img.add_to(m)

folium.LayerControl().add_to(m)
m